In [2]:
import cv2
import numpy as np
import mediapipe as mp
import time
import os

In [4]:
folderPath = "Header"
myList = os.listdir(folderPath)
print(myList)
overLayList = []
for imPath in myList:
    image = cv2.imread('{folderPath}/{imPath}')
    overLayList.append(image)
print(len(overLayList))

[]
0


In [24]:

class handDetector:
    def __init__(self, mode=False, maxHands=2, detectionCon=0.5, trackCon=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.trackCon = trackCon
        
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(
            static_image_mode=self.mode,
            max_num_hands=self.maxHands,
            min_detection_confidence=float(self.detectionCon),
            min_tracking_confidence=float(self.trackCon)
        )
        self.mpDraw = mp.solutions.drawing_utils
        self.tipIds=[4,8,12,16,20]
    def findHands(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms, self.mpHands.HAND_CONNECTIONS)
        return img

    def findPosition(self, img, handNo=0, draw= True ):
        self.lmList = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo] 
            for id, lm in enumerate(myHand.landmark):
                # print(id,lm)
                h, w, c =img.shape
                cx, cy = int(lm.x * w),int(lm.y * h)
                lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy),15, (255,0,255), cv2.FILLED)      
            return self.lmList
    
    def fingersUp(self):
        fingers=[]
        #Thumb
        if lmList[self.tipIds[0]][1] > lmList[self.tipIds[0]-1][1]:
            fingers.append(1)
        else:
            fingers.append(0)
        # 4 Fingers
        for id in range(1,5):
            if lmList[self.tipIds[id]][2]< lmList[self.tipIds[id]-2][2]:
                fingers.append(1)
            else:
                fingers.append(0)
        

In [22]:
# header = overLayList[0]

cap = cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)

detector = handDetector(detectionCon = 0.85)

while True:

        # 1. Import Image
        success, img = cap.read()
        img = cv2.flip(img,1)
    
        # 2. Find Hand Landmarks
        img=detector.findHands(img)
        lmList=detector.findPosition(img,draw=False)
        if lmList is not None and len(lmList) > 0:
            print(lmList)
            #tip of index and middle fingers
            x1,y1=lmList[8][1:]
            x2,y2=lmList[12][1:]
    
        # 3. Check which fingers are up.
        # 4. If Selection mode - Two fingers are up.
        # 5. If Selection mode - Index finger is up.
    
        # Setting  the header image
        # img[0:125,0:1280] = header
        cv2.imshow("Image" , img)
        if cv2.waitKey(1)& 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

[[0, 543, 726], [1, 474, 687], [2, 433, 626], [3, 413, 573], [4, 401, 529], [5, 505, 531], [6, 498, 457], [7, 491, 409], [8, 488, 370], [9, 550, 528], [10, 551, 442], [11, 546, 385], [12, 540, 339], [13, 593, 544], [14, 604, 460], [15, 605, 406], [16, 605, 359], [17, 633, 575], [18, 655, 514], [19, 665, 472], [20, 672, 432]]
[[0, 572, 718], [1, 501, 680], [2, 458, 614], [3, 430, 560], [4, 413, 512], [5, 529, 510], [6, 518, 431], [7, 506, 384], [8, 494, 343], [9, 578, 507], [10, 581, 418], [11, 576, 361], [12, 569, 314], [13, 624, 523], [14, 638, 436], [15, 640, 380], [16, 638, 330], [17, 665, 555], [18, 693, 491], [19, 706, 446], [20, 716, 404]]
[[0, 606, 709], [1, 528, 672], [2, 478, 602], [3, 448, 543], [4, 427, 493], [5, 552, 489], [6, 541, 407], [7, 532, 357], [8, 525, 314], [9, 605, 485], [10, 606, 391], [11, 603, 333], [12, 600, 284], [13, 654, 501], [14, 667, 410], [15, 670, 354], [16, 671, 303], [17, 699, 534], [18, 726, 466], [19, 741, 422], [20, 753, 379]]
[[0, 677, 673], [1,